# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [180]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
import xlsxwriter

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [181]:
stocks = pd.read_csv('sp_500_stocks_v1.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [182]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2218935946336,
 'week52high': 138.42,
 'week52low': 55.98,
 'week52change': 0.8690767869885053,
 'sharesOutstanding': 17531495733,
 'float': 0,
 'avg10Volume': 99131806,
 'avg30Volume': 98299744,
 'day200MovingAvg': 111.96,
 'day50MovingAvg': 121.91,
 'employees': 0,
 'ttmEPS': 3.41,
 'ttmDividendRate': 0.8109590832297764,
 'dividendYield': 0.006521097262063128,
 'nextDividendDate': '0',
 'exDividendDate': '2020-10-24',
 'nextEarningsDate': '0',
 'peRatio': 38.00055600773243,
 'beta': 1.2073616379109136,
 'maxChangePercent': 50.20226108770235,
 'year5ChangePercent': 4.158335561570921,
 'year2ChangePercent': 2.16663481697574,
 'year1ChangePercent': 0.9186350906787538,
 'ytdChangePercent': 0.758444748474264,
 'month6ChangePercent': 0.502694987855581,
 'month3ChangePercent': 0.11143963665964515,
 'month1ChangePercent': 0.07438083331416433,
 'day30ChangePercent': 0.07502875569346029,
 'day5ChangePercent': 0.0289816422566698}

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [183]:
data['year1ChangePercent']

0.9186350906787538

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [184]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    #print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [185]:
final_dataframe = pd.DataFrame(columns = my_columns)
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
#     print(data['AAPL']['stats'])
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series(
            [
                symbol,
                data[symbol]['price'],
                data[symbol]['stats']['year1ChangePercent'],
                'N/A',
                
            ],
            index = my_columns),
            ignore_index = True
        )
final_dataframe


,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,119.670,0.432582,N/A
1,AAL,17.120,-0.395937,N/A
2,AAP,162.368,0.037677,N/A
3,AAPL,129.720,0.881610,N/A
4,ABBV,109.220,0.245950,N/A
...,...,...,...,...
496,YUM,110.820,0.094847,N/A
497,ZBH,156.850,0.006434,N/A
498,ZBRA,387.270,0.494783,N/A
499,ZION,41.777,-0.156623,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [186]:
final_dataframe.sort_values('One-Year Price Return',ascending = False, inplace = True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe


,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,78,CARR,38.908,2.212926,N/A
1,341,NVDA,544.700,1.447462,N/A
2,31,AMD,100.460,1.410074,N/A
3,273,LB,39.650,1.296779,N/A
4,23,ALB,136.640,1.145290,N/A
5,383,PYPL,232.090,1.092016,N/A
6,335,NOW,555.000,0.967279,N/A
7,24,ALGN,524.540,0.942748,N/A
8,177,FCX,24.526,0.920760,N/A
9,3,AAPL,129.720,0.881610,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [187]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the size of your portfolio: ")
    
    try:
        float(portfolio_size)
    except:
        print("That is not a number! \nPlease try again")
        portfolio_size = input("Enter the size of your portfolio: ")
portfolio_input()
print(portfolio_size)

Enter the size of your portfolio: 10000000
10000000


In [188]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(len(final_dataframe)):
    final_dataframe.loc[i, "Number of Shares to Buy"] = math.floor(position_size / final_dataframe.loc[i, "Price"])
final_dataframe

/Users/parikshitdubey/.conda/envs/Maps_Hashing/lib/python3.8/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,78,CARR,38.908,2.212926,5140
1,341,NVDA,544.700,1.447462,367
2,31,AMD,100.460,1.410074,1990
3,273,LB,39.650,1.296779,5044
4,23,ALB,136.640,1.145290,1463
5,383,PYPL,232.090,1.092016,861
6,335,NOW,555.000,0.967279,360
7,24,ALGN,524.540,0.942748,381
8,177,FCX,24.526,0.920760,8154
9,3,AAPL,129.720,0.881610,1541


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [189]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM Score'
    
]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)
hqm_dataframe

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['price'],
                    'N/A',
                    data[symbol]['stats']['year1ChangePercent'],
                    'N/A',
                    data[symbol]['stats']['month6ChangePercent'],
                    'N/A',
                    data[symbol]['stats']['month3ChangePercent'],
                    'N/A',
                    data[symbol]['stats']['month1ChangePercent'],
                    'N/A',
                    'N/A'
                ],
                index = hqm_columns),
            ignore_index = True
        )

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,119.470,N/A,0.422491,N/A,0.361247,N/A,0.172668,N/A,0.068768,N/A,N/A
1,AAL,17.220,N/A,-0.395465,N/A,0.019014,N/A,0.309103,N/A,0.392020,N/A,N/A
2,AAP,165.520,N/A,0.037510,N/A,0.162632,N/A,0.020976,N/A,0.039093,N/A,N/A
3,AAPL,131.614,N/A,0.876327,N/A,0.508143,N/A,0.111329,N/A,0.074472,N/A,N/A
4,ABBV,109.300,N/A,0.245365,N/A,0.138027,N/A,0.151993,N/A,0.039279,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,111.870,N/A,0.097282,N/A,0.188204,N/A,0.155206,N/A,0.049575,N/A,N/A
497,ZBH,156.695,N/A,0.006223,N/A,0.170067,N/A,0.027091,N/A,-0.023762,N/A,N/A
498,ZBRA,392.050,N/A,0.502337,N/A,0.483346,N/A,0.496114,N/A,0.113353,N/A,N/A
499,ZION,41.701,N/A,-0.159919,N/A,0.180101,N/A,0.357855,N/A,0.115055,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [190]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_column = f'{time_period} Return Percentile'
        hqm_dataframe.loc[row, percentile_column] = stats.percentileofscore(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col])/100


# Print each percentile score to make sure it was calculated properly
for time_period in time_periods:
    print(hqm_dataframe[f'{time_period} Return Percentile'])

#Print the entire DataFrame    
hqm_dataframe



0       0.868263
1      0.0279441
2       0.409182
3       0.982036
4       0.712575
         ...    
496     0.510978
497     0.363273
498     0.908184
499     0.143713
500     0.786427
Name: One-Year Return Percentile, Length: 501, dtype: object
0      0.762475
1      0.123752
2      0.377246
3      0.906188
4      0.325349
         ...   
496    0.431138
497    0.393214
498    0.878244
499    0.411178
500    0.518962
Name: Six-Month Return Percentile, Length: 501, dtype: object
0      0.590818
1      0.860279
2      0.217565
3      0.433134
4      0.532934
         ...   
496    0.542914
497    0.237525
498    0.954092
499    0.896208
500    0.163673
Name: Three-Month Return Percentile, Length: 501, dtype: object
0      0.650699
1      0.988024
2      0.518962
3      0.672655
4      0.522954
         ...   
496    0.562874
497    0.183633
498    0.788423
499    0.792415
500    0.179641
Name: One-Month Return Percentile, Length: 501, dtype: object


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,119.470,N/A,0.422491,0.868263,0.361247,0.762475,0.172668,0.590818,0.068768,0.650699,N/A
1,AAL,17.220,N/A,-0.395465,0.0279441,0.019014,0.123752,0.309103,0.860279,0.392020,0.988024,N/A
2,AAP,165.520,N/A,0.037510,0.409182,0.162632,0.377246,0.020976,0.217565,0.039093,0.518962,N/A
3,AAPL,131.614,N/A,0.876327,0.982036,0.508143,0.906188,0.111329,0.433134,0.074472,0.672655,N/A
4,ABBV,109.300,N/A,0.245365,0.712575,0.138027,0.325349,0.151993,0.532934,0.039279,0.522954,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,111.870,N/A,0.097282,0.510978,0.188204,0.431138,0.155206,0.542914,0.049575,0.562874,N/A
497,ZBH,156.695,N/A,0.006223,0.363273,0.170067,0.393214,0.027091,0.237525,-0.023762,0.183633,N/A
498,ZBRA,392.050,N/A,0.502337,0.908184,0.483346,0.878244,0.496114,0.954092,0.113353,0.788423,N/A
499,ZION,41.701,N/A,-0.159919,0.143713,0.180101,0.411178,0.357855,0.896208,0.115055,0.792415,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [191]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,119.470,N/A,0.422491,0.868263,0.361247,0.762475,0.172668,0.590818,0.068768,0.650699,0.718064
1,AAL,17.220,N/A,-0.395465,0.0279441,0.019014,0.123752,0.309103,0.860279,0.392020,0.988024,0.5
2,AAP,165.520,N/A,0.037510,0.409182,0.162632,0.377246,0.020976,0.217565,0.039093,0.518962,0.380739
3,AAPL,131.614,N/A,0.876327,0.982036,0.508143,0.906188,0.111329,0.433134,0.074472,0.672655,0.748503
4,ABBV,109.300,N/A,0.245365,0.712575,0.138027,0.325349,0.151993,0.532934,0.039279,0.522954,0.523453
...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,111.870,N/A,0.097282,0.510978,0.188204,0.431138,0.155206,0.542914,0.049575,0.562874,0.511976
497,ZBH,156.695,N/A,0.006223,0.363273,0.170067,0.393214,0.027091,0.237525,-0.023762,0.183633,0.294411
498,ZBRA,392.050,N/A,0.502337,0.908184,0.483346,0.878244,0.496114,0.954092,0.113353,0.788423,0.882236
499,ZION,41.701,N/A,-0.159919,0.143713,0.180101,0.411178,0.357855,0.896208,0.115055,0.792415,0.560878


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [192]:
hqm_dataframe.sort_values(by = 'HQM Score', ascending = False, inplace = True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(inplace = True, drop = True)
hqm_dataframe


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,FCX,25.196,N/A,0.923444,0.984032,1.327688,0.998004,0.499871,0.96008,0.215255,0.914172,0.964072
1,LRCX,505.820,N/A,0.814802,0.972056,0.727713,0.964072,0.639206,0.986028,0.169002,0.870259,0.948104
2,ALGN,525.622,N/A,0.968657,0.986028,1.002960,0.992016,0.548716,0.976048,0.134932,0.834331,0.947106
3,AMAT,89.550,N/A,0.514244,0.918164,0.564532,0.938124,0.552500,0.978044,0.224602,0.928144,0.940619
4,XLNX,156.990,N/A,0.622833,0.94012,0.706656,0.962076,0.501961,0.962076,0.200235,0.898204,0.940619
5,LB,40.180,N/A,1.287429,0.994012,1.583379,1,0.367358,0.908184,0.147483,0.846307,0.937126
6,ALB,137.940,N/A,1.146302,0.992016,0.839657,0.984032,0.428092,0.936128,0.129327,0.824351,0.934132
7,TWTR,55.250,N/A,0.748299,0.958084,0.555428,0.932136,0.367926,0.912176,0.222530,0.924152,0.931637
8,MU,74.290,N/A,0.445129,0.884232,0.492682,0.888224,0.495036,0.952096,0.274681,0.96008,0.921158
9,URI,233.830,N/A,0.474447,0.896208,0.626145,0.952096,0.361821,0.9002,0.157263,0.856287,0.901198


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [193]:
portfolio_input()

Enter the size of your portfolio: 10000000


In [194]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)
for row in hqm_dataframe.index:
    hqm_dataframe.loc[row, "Number of Shares to Buy"] = math.floor(position_size/hqm_dataframe.loc[row, 'Price'])
hqm_dataframe

/Users/parikshitdubey/.conda/envs/Maps_Hashing/lib/python3.8/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,FCX,25.196,7937,0.923444,0.984032,1.327688,0.998004,0.499871,0.96008,0.215255,0.914172,0.964072
1,LRCX,505.820,395,0.814802,0.972056,0.727713,0.964072,0.639206,0.986028,0.169002,0.870259,0.948104
2,ALGN,525.622,380,0.968657,0.986028,1.002960,0.992016,0.548716,0.976048,0.134932,0.834331,0.947106
3,AMAT,89.550,2233,0.514244,0.918164,0.564532,0.938124,0.552500,0.978044,0.224602,0.928144,0.940619
4,XLNX,156.990,1273,0.622833,0.94012,0.706656,0.962076,0.501961,0.962076,0.200235,0.898204,0.940619
5,LB,40.180,4977,1.287429,0.994012,1.583379,1,0.367358,0.908184,0.147483,0.846307,0.937126
6,ALB,137.940,1449,1.146302,0.992016,0.839657,0.984032,0.428092,0.936128,0.129327,0.824351,0.934132
7,TWTR,55.250,3619,0.748299,0.958084,0.555428,0.932136,0.367926,0.912176,0.222530,0.924152,0.931637
8,MU,74.290,2692,0.445129,0.884232,0.492682,0.888224,0.495036,0.952096,0.274681,0.96008,0.921158
9,URI,233.830,855,0.474447,0.896208,0.626145,0.952096,0.361821,0.9002,0.157263,0.856287,0.901198


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [195]:
writer = pd.ExcelWriter('MomentumStrategy.xlsx', engine = 'xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [196]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [197]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', integer_template]
                    }
for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [198]:
writer.save()